In [2]:
import pyteomics
import lxml
from pyteomics import mzml, mass
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from matplotlib import pyplot
import matplotlib
import seaborn as sns
import re
import math
import statsmodels
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [2]:
sample=[1,1,2,2]
def remove_dul(tar_list=sample):
    res=[]
    [res.append(x) for x in tar_list if x not in res]
    return(res)
def procount(dir='D:\\csodiaq'):
    plasma_pro = []
    for file in os.listdir(dir):
        if file.endswith(".csv") and 'proteinFDR' in file:
            plasma_pro.append(os.path.join(dir, file))
    dic={}
    for i in range(len(plasma_pro)):
        df=pd.read_csv(plasma_pro[i]) 
        yummy=len(remove_dul(tar_list=df['protein'].tolist()))
        dic[str(plasma_pro[i].split('\\')[-1])]=yummy
        dfnew=pd.DataFrame.from_dict(dic, orient='index')
    return(dfnew)

In [10]:
# import the raw values of common proteins from multiomic methods 
df_pro=pd.read_csv(r'D:\\yuming\\2023\\20230309\\csodiaq_quan\\common_proteins_macro.csv')


In [ ]:
df_pro=df_pro.iloc[:,:19]
df_pro.set_index('Unnamed: 0',inplace=True)
df_pro

In [ ]:
dfpro_log2=np.log2(df_pro)
#dfpro_log2=df_pro
dfpro_log2

In [ ]:
# apply standard scaler to the new dataframe 
ss = StandardScaler() 
df_ss = ss.fit_transform(dfpro_log2)
dfpro_std=pd.DataFrame(df_ss,index=dfpro_log2.index,columns=dfpro_log2.columns)
dfpro_std

In [14]:
# function for oneway_ANOVA analysis 
def oneway_ANOVA(result=dfpro_std):
    pval=[]
    for i in range(len(result)):
        aa=result.iloc[i,:6].tolist()
        bb=result.iloc[i,6:12].tolist()
        cc=result.iloc[i,12:18].tolist()     
        p_value=f_oneway(aa,bb,cc)[1]
        pval.append([i,p_value])
        fl = pd.DataFrame(pval)
        fl.columns = ['number', 'p_value']
    return(fl)

In [ ]:
# apply anova to all proteins, and calculate p VALUES
dfpro_pval=oneway_ANOVA(result=dfpro_std)
dfpro_pval['index_name']=dfpro_std.index.values.tolist()
dfpro_pval

In [ ]:
len([item for item in dfpro_pval['p_value'].tolist() if item <0.05])

In [10]:
FDR_res=statsmodels.stats.multitest.fdrcorrection(dfpro_pval['p_value'].tolist(), alpha=0.05, method='indep', is_sorted=False)[0]

In [ ]:
# calculate the pvalues < 0.05 after FDR correction 

yuyu=[]
for item in range(len(FDR_res)):
    if FDR_res[item]== True:
        yuyu.append(item)
len(yuyu)

In [ ]:
# locate the significant proteins from df_pval dataframe, aiming to find the protein names out 
sig_pro=dfpro_pval.loc[yuyu]
sig_pro

In [ ]:
dfpro_sig=dfpro_std.loc[sig_pro['index_name']]
dfpro_sig

In [88]:
k=[item.split("|")[1] for item in dfpro_sig.index]
dfk=pd.DataFrame(k)


In [14]:
treatment=['Con']*6+['LPS']*6+['IL_4']*6
PCA_pro_sig=dfpro_sig.T

In [ ]:
# set the data soucrce
df =PCA_pro_sig

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)

# perform PCA with 2 principal components
pca = PCA(n_components=2)
pca.fit(X_scaled)

# transform the features to the new space
X_pca = pca.transform(X_scaled)

# create a new dataframe with the principal components and target variable
df_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])
df_pca['target'] = treatment

# plot the results
import seaborn as sns
plt.figure(figsize=(8, 6), edgecolor='#04253a')
sns.scatterplot(x='PC1', y='PC2', hue='target', data=df_pca, s=180)
plt.yticks(fontsize=22)
plt.xticks(fontsize=22)
plt.xlabel('PC 1', fontsize=22)
plt.ylabel('PC 2', fontsize=22)
plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0.2,fontsize=20)

In [16]:
dfpro_std2=dfpro_std

In [ ]:
# standardize by rows
scaler=StandardScaler()
proname=dfpro_std2.T.columns
k=dfpro_std2.T
dfdf_pro=ss.fit_transform(k)

dfpro_stdbyrow=pd.DataFrame(dfdf_pro.T,index=dfpro_std2.index,columns=dfpro_std2.columns)
dfpro_stdbyrow

In [ ]:
#calculate the mean value of proteins from 6 repeats in each treatment 
con=dfpro_stdbyrow.iloc[:,:6].mean(axis=1).tolist()
lps=dfpro_stdbyrow.iloc[:,6:12].mean(axis=1).tolist()
IL_4=dfpro_stdbyrow.iloc[:,12:18].mean(axis=1).tolist()
newdict={'Con':con,'LPS':lps,'IL_4':IL_4}

dfpro_mean=pd.DataFrame.from_dict(newdict)
dfpro_mean.index=dfpro_stdbyrow.index
dfpro_mean

In [19]:
sigpro_stdrow=dfpro_mean.loc[sig_pro['index_name']]

In [37]:
def optimise_k_means(data,max_k):
    means=[]
    inertias=[]
    for k in range(1,max_k):
        kmeans=KMeans(n_clusters=k)
        kmeans.fit(data)
        means.append(k)
        inertias.append(kmeans.inertia_)
    fig=plt.subplots(figsize=(10,5))
    plt.plot(means,inertias)
    plt.xlabel('number of clusters')
    plt.ylabel('inertia')
    plt.show()

In [24]:
optimise_k_means(sigpro_stdrow,10)

In [ ]:
kmeans=KMeans(n_clusters=4)
kmeans.fit(sigpro_stdrow[['Con','LPS','IL_4']])

sigpro_stdrow['kmeans']=kmeans.labels_
sigpro_stdrow

In [ ]:
# plot heatmap for significant metabolites
df_df=sigpro_stdrow
dfaa=df_df.sort_values(by='kmeans')
df_bb=dfaa.T
df_heat=df_bb.iloc[:3,:]

plt.figure(figsize=(22,5),edgecolor="#04253a")
matplotlib.rcParams['font.family'] = "Arial"
sns.heatmap(df_heat,  vmin=-1.5, vmax=1.5,cmap='viridis',xticklabels=False,cbar_kws={"shrink": 1, 'ticks': [-2, -1, 0, 1, 2]})
plt.xticks([])
plt.yticks()
plt.tick_params(axis='y',length=6)
plt.yticks(rotation=90)
plt.yticks(fontsize=22)
plt.savefig(r'D:\\project7_SMAD\\SI_figures\\sigpro_heatmap.svg',dpi=1000,bbox_inches='tight')

In [22]:
x=['Con','LPS','IL_4']

# take different cluster protein names out
#sig_cluster4=sigpro_stdrow.loc[sigpro_stdrow['kmeans']==4]
sig_cluster3=sigpro_stdrow.loc[sigpro_stdrow['kmeans']==3]
sig_cluster2=sigpro_stdrow.loc[sigpro_stdrow['kmeans']==2]
sig_cluster1=sigpro_stdrow.loc[sigpro_stdrow['kmeans']==1]
sig_cluster0=sigpro_stdrow.loc[sigpro_stdrow['kmeans']==0]

In [ ]:
import matplotlib.pyplot as plt

# Create a dictionary of cluster names and corresponding data
cluster_data = {'sigpro_Cluster 1': sigpro_stdrow.loc[list(sig_cluster0.index)],
                'sigpro_Cluster 2': sigpro_stdrow.loc[list(sig_cluster1.index)],
                'sigpro_Cluster 3': sigpro_stdrow.loc[list(sig_cluster2.index)],
                'sigpro_Cluster 4': sigpro_stdrow.loc[list(sig_cluster3.index)],}
                #'Cluster 5': sigpro_stdrow.loc[list(sig_cluster4.index)]}

# Loop over the dictionary items to plot the data for each cluster
for title, data in cluster_data.items():
    plt.figure(figsize=(6, 5), edgecolor='#04253a')
    plt.rcParams['font.family'] = 'Arial'
    for i in range(len(data)):
        plt.plot(x, data.iloc[i, :3])  
    #plt.title(title, fontsize=24)
    plt.xlabel('Treatment', fontsize=24)
    plt.ylabel('Score', fontsize=24)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=22)
    plt.savefig(r'D:\\project7_SMAD\\SI_figures\\{}.svg'.format(title),dpi=800,bbox_inches='tight')
    plt.show()

# metabolome analysis of macrophage starts from here

In [4]:
# import raw data of macrophage metabolome 

df_meta=pd.read_excel(r'E:\\project7_SMAD\\raw_data\\macro_metabolites_raw.xlsx')

In [ ]:
df_meta=df_meta.drop_duplicates(subset=['mz'],keep='first')
df_meta.set_index('mz',inplace=True)
df_meta

In [ ]:
dfmeta_log2=np.log2(df_meta)
#dfpro_log2=df_pro
dfmeta_log2

In [ ]:
ss = StandardScaler() 
dfmeta_ss = ss.fit_transform(dfmeta_log2)
dfmeta_std=pd.DataFrame(dfmeta_ss,index=dfmeta_log2.index,columns=dfmeta_log2.columns)
dfmeta_std

In [ ]:
# calculate the p values of metabolome for different treatments 

dfmeta_pval=oneway_ANOVA(result=dfmeta_std)
dfmeta_pval['index_name']=dfmeta_std.index.values.tolist()
dfmeta_pval

In [ ]:
len([item for item in dfmeta_pval['p_value'].tolist() if item <0.05])

In [ ]:
FDR_res_meta=statsmodels.stats.multitest.fdrcorrection(dfmeta_pval['p_value'].tolist(), alpha=0.05, method='indep', is_sorted=False)[0]

In [ ]:
# calculate the pvalues < 0.01 after FDR correction 

meta_yuyu=[]
for item in range(len(FDR_res_meta)):
    if FDR_res_meta[item]== True:
        meta_yuyu.append(item)
len(meta_yuyu)

In [ ]:
# locate the significant m/z from dfmeta_std dataframe, aiming to find the sig mz names out 

sig_meta=dfmeta_pval.loc[meta_yuyu]
sig_meta

In [ ]:
dfmeta_sig=dfmeta_std.loc[sig_meta['index_name']]
dfmeta_sig

In [35]:
treatment=['Con']*6+['LPS']*6+['IL_4']*6
PCA_meta_sig=dfmeta_sig.T

In [ ]:
# set the data soucrce
df =PCA_meta_sig
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)

# perform PCA with 2 principal components
pca = PCA(n_components=2)
pca.fit(X_scaled)

# transform the features to the new space
X_pca = pca.transform(X_scaled)

# create a new dataframe with the principal components and target variable
df_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])
df_pca['target'] = treatment

# plot the results
plt.figure(figsize=(8, 6), edgecolor='#04253a')
sns.scatterplot(x='PC1', y='PC2', hue='target', data=df_pca, s=180)
plt.yticks(fontsize=22)
plt.xticks(fontsize=22)
plt.xlabel('PC 1', fontsize=22)
plt.ylabel('PC 2', fontsize=22)
plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0.2,fontsize=20)

In [37]:
#K means clustering process of significant metabolite features start here

In [21]:
dfmeta_std2=dfmeta_std

In [ ]:
# standardize by rows
scaler=StandardScaler()
metaname=dfmeta_std2.T.columns
k=dfmeta_std2.T
dfdf_meta=ss.fit_transform(k)

dfmeta_stdbyrow=pd.DataFrame(dfdf_meta.T,index=dfmeta_std2.index,columns=dfmeta_std2.columns)
dfmeta_stdbyrow

In [ ]:
#calculate the mean value of metateins from 6 repeats in each treatment 
con=dfmeta_stdbyrow.iloc[:,:6].mean(axis=1).tolist()
lps=dfmeta_stdbyrow.iloc[:,6:12].mean(axis=1).tolist()
IL_4=dfmeta_stdbyrow.iloc[:,12:18].mean(axis=1).tolist()
newdict={'Con':con,'LPS':lps,'IL_4':IL_4}

dfmeta_mean=pd.DataFrame.from_dict(newdict)
dfmeta_mean.index=dfmeta_stdbyrow.index
dfmeta_mean

In [ ]:
sigmeta_stdrow=dfmeta_mean.loc[sig_meta['index_name']]
sigmeta_stdrow

In [ ]:
optimise_k_means(sigmeta_stdrow,10)

In [ ]:
kmeans=KMeans(n_clusters=4)
kmeans.fit(sigmeta_stdrow[['Con','LPS','IL_4']])

sigmeta_stdrow['kmeans']=kmeans.labels_
sigmeta_stdrow

In [ ]:
plt.rcParams['font.family'] = 'Arial'
for i in [0,1,2,3]:
    plt.figure(figsize=(6, 5), edgecolor='#04253a')
    plt.hist(sigmeta_stdrow[sigmeta_stdrow['kmeans']==i].index,bins=50) 
#plt.title(title, fontsize=24)
    plt.xlabel('m/z', fontsize=24)
    plt.ylabel('Counts', fontsize=24)
    plt.xticks(fontsize=22)
    plt.yticks(fontsize=22)
    plt.savefig(r'D:\\project7_SMAD\\SI_figures\\{}.svg'.format(i),dpi=800,bbox_inches='tight')
    plt.show()

In [ ]:
# plot heatmap for significant metabolites
df_df=sigmeta_stdrow
dfaa=df_df.sort_values(by='kmeans')
df_bb=dfaa.T
df_heat=df_bb.iloc[:3,:]

plt.figure(figsize=(22,5),edgecolor="#04253a")
matplotlib.rcParams['font.family'] = "Arial"
sns.heatmap(df_heat,  vmin=-1.5, vmax=1.5,cmap='viridis',xticklabels=False,cbar_kws={"shrink": 1, 'ticks': [-2, -1, 0, 1, 2]})
plt.xticks([])
plt.yticks()
plt.tick_params(axis='y',length=6)
plt.yticks(rotation=90)
plt.yticks(fontsize=22)
plt.savefig(r'D:\\project7_SMAD\\figure_3_macro_clustering_analysis\\SI_figures\\sigmeta_heatmap.svg',dpi=600,bbox_inches='tight')

In [34]:
x=['Con','LPS','IL_4']

# take different cluster metatein names out
#sig_cluster4=sigmeta_stdrow.loc[sigmeta_stdrow['kmeans']==4]
sig_cluster3=sigmeta_stdrow.loc[sigmeta_stdrow['kmeans']==3]
sig_cluster2=sigmeta_stdrow.loc[sigmeta_stdrow['kmeans']==2]
sig_cluster1=sigmeta_stdrow.loc[sigmeta_stdrow['kmeans']==1]
sig_cluster0=sigmeta_stdrow.loc[sigmeta_stdrow['kmeans']==0]

In [ ]:
import matplotlib.pyplot as plt

# Create a dictionary of cluster names and corresponding data
cluster_data = {'sigmeta_Cluster 1': sigmeta_stdrow.loc[list(sig_cluster0.index)],
                'sigmeta_Cluster 2': sigmeta_stdrow.loc[list(sig_cluster1.index)],
                'sigmeta_Cluster 3': sigmeta_stdrow.loc[list(sig_cluster2.index)],
                'sigmeta_Cluster 4': sigmeta_stdrow.loc[list(sig_cluster3.index)],}
                #'Cluster 5': sigmeta_stdrow.loc[list(sig_cluster4.index)]}

# Loop over the dictionary items to plot the data for each cluster
for title, data in cluster_data.items():
    plt.figure(figsize=(6, 5), edgecolor='#04253a')
    plt.rcParams['font.family'] = 'Arial'
    for i in range(len(data)):
        plt.plot(x, data.iloc[i, :3])  
    plt.title(title, fontsize=24)
    #plt.xlabel('Treatment', fontsize=24)
    plt.ylabel('Score', fontsize=24)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=22)
    plt.savefig(r'D:\\project7_SMAD\\figure_3_macro_clustering_analysis\\SI_figures\\{}.svg'.format(title),dpi=800,bbox_inches='tight')
    plt.show()

In [52]:
# Trying to match significant meta m/z to the identified m/z

In [ ]:
# input the identified m/z features from GNPS results 
fileslist = []
dir = 'D:\\project7_SMAD\\meta_identification'
for file in os.listdir(dir):
    if file.endswith(".tsv"):
        fileslist.append(os.path.join(dir, file))
print(fileslist)

In [4]:
meta_macro=pd.read_csv(fileslist[1],delimiter="\t", header=0)
# filter the dataframe with 'MZErrorPPM' < 30 ppm

meta_macro_filter=meta_macro[meta_macro['MZErrorPPM'] < 30]

In [57]:
#len(remove_dul(tar_list=meta_macro_filter['Compound_Name'].tolist()))

In [9]:
# try to match the significant metabolite m/z features to the identified m/z features from GNPS

scan_20_25_30=meta_macro_filter[meta_macro_filter['full_CCMS_path']=='/ST_Macro_mix2_meta_DDA_Qslices_20_25_30.mzXML']
scan_35_40_45=meta_macro_filter[meta_macro_filter['full_CCMS_path']=='/ST_Macro_mix2_meta_DDA_Qslices_35_40_45.mzXML']
scan_5_10_15=meta_macro_filter[meta_macro_filter['full_CCMS_path']=='/ST_Macro_mix2_meta_DDA_Qslices_5_10_15.mzXML']

In [ ]:
mzmlslist = []
dir = 'D:\\ST_MACRO'
for file in os.listdir(dir):
    if file.endswith(".mzML") and 'Macro' in file:
        mzmlslist.append(os.path.join(dir, file))
print(mzmlslist)

In [12]:
tmp_5_10_15 = pyteomics.mzml.read(mzmlslist[1], use_index=True)
tmp_20_25_30 = pyteomics.mzml.read(mzmlslist[2], use_index=True)
tmp_35_40_45 = pyteomics.mzml.read(mzmlslist[0], use_index=True)

In [13]:
list_cv1,list_cv2,list_cv3=[],[],[]

for number in list(np.array(scan_5_10_15['#Scan#'].tolist())-1):
    list_cv1.append(float(tmp_5_10_15.get_by_index(number)['precursorList']['precursor'][0]['isolationWindow']['isolation window target m/z']))
for number in list(np.array(scan_20_25_30['#Scan#'].tolist())-1):
    list_cv2.append(float(tmp_20_25_30.get_by_index(number)['precursorList']['precursor'][0]['isolationWindow']['isolation window target m/z']))
for number in list(np.array(scan_35_40_45['#Scan#'].tolist())-1):
    list_cv3.append(float(tmp_35_40_45.get_by_index(number)['precursorList']['precursor'][0]['isolationWindow']['isolation window target m/z']))

In [ ]:
new_df1=scan_5_10_15.iloc[:,:7]
new_df2=scan_20_25_30.iloc[:,:7]
new_df3=scan_35_40_45.iloc[:,:7]
new_df1['mz']=list_cv1
new_df2['mz']=list_cv2
new_df3['mz']=list_cv3
newdf_all=pd.concat([new_df1,new_df2,new_df3])
newdf_all

In [ ]:
newdf_all['mz_rounded']=[round(num, 3) for num in newdf_all['mz']]
newdf_all

In [16]:
newdf_all.to_excel(r'D:\\project7_SMAD\\GNPS_RESULTS.xlsx')

In [ ]:
mz=[round(x, 3) for x in dfmeta_sig.index.tolist()]

dfmeta_sig2=dfmeta_sig
dfmeta_sig2['mz_rounded']=mz
dfmeta_sig2

In [66]:
# get the significant m/z and corresponding compound name out  named the dataframe to name_df

name_df=dfmeta_sig2[dfmeta_sig2['mz_rounded'].isin (newdf_all['mz_rounded'].tolist())]


In [ ]:
name_df.to_excel(r'D:\\project7_SMAD\\macro_sigmeta_identified.xlsx')
name_df

In [68]:
dfmeta_id=pd.read_excel(r'D:\\project7_SMAD\\sig_meta_identified.xlsx')

In [69]:
dfmeta_id.set_index('mz_rounded',inplace=True)

In [ ]:
dfmeta_id=dfmeta_id.iloc[:,1:]
dfmeta_id

In [ ]:
# K means clustering of the identifed and significat metabolite features 

sss=StandardScaler()
anyte=dfmeta_id.T.columns
er=dfmeta_id.T
dfer=ss.fit_transform(er)

dfer_stdbyrow=pd.DataFrame(dfer.T,index=dfmeta_id.index,columns=dfmeta_id.columns)
dfer_stdbyrow


In [ ]:
#calculate the mean value of identified metabolites of 6 repeats for each treatment, socurce dataframe is after two times std_scaler 
con2=dfer_stdbyrow.iloc[:,:6].mean(axis=1).tolist()
lps2=dfer_stdbyrow.iloc[:,6:12].mean(axis=1).tolist()
IL_42=dfer_stdbyrow.iloc[:,12:18].mean(axis=1).tolist()
newdict2={'Con':con2,'LPS':lps2,'IL_4':IL_42}

dfer_mean=pd.DataFrame.from_dict(newdict2)
dfer_mean.index=dfer_stdbyrow.index
dfer_mean

In [ ]:
kmeans=KMeans(n_clusters=4)
kmeans.fit(dfer_mean[['Con','LPS','IL_4']])

dfer_mean['kmeans']=kmeans.labels_
dfer_mean

In [75]:
sample=[2,4,6,8,3]
def remove_dul(tar_list=sample):
    res=[]
    [res.append(x) for x in tar_list if x not in res]
    return(res)

def plot_clusters(df=sigmeta_stdrow):
    x=['Con','LPS','IL_4']
    # take different cluster metatein names out
    #sig_cluster4=df.loc[df['kmeans']==4]
    sig_cluster3=df.loc[df['kmeans']==3]
    sig_cluster2=df.loc[df['kmeans']==2]
    sig_cluster1=df.loc[df['kmeans']==1]
    sig_cluster0=df.loc[df['kmeans']==0]
    # Create a dictionary of cluster names and corresponding data
    cluster_data = {'Cluster 1': df.loc[list(sig_cluster0.index)],
                    'Cluster 2': df.loc[list(sig_cluster1.index)],
                    'Cluster 3': df.loc[list(sig_cluster2.index)],
                    'Cluster 4': df.loc[list(sig_cluster3.index)],}
                    #'Cluster 5': df.loc[list(sig_cluster4.index)]}

    # Loop over the dictionary items to plot the data for each cluster
    for title, data in cluster_data.items():
        plt.figure(figsize=(6, 5), edgecolor='#04253a')
        plt.rcParams['font.family'] = 'Arial'
        for i in range(len(data)):
            plt.plot(x, data.iloc[i, :3])  
        plt.title(title, fontsize=24)
        plt.xlabel('Treatment', fontsize=24)
        plt.ylabel('Score', fontsize=24)
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=22)
        plt.show()

In [76]:
plot_clusters(df=dfer_mean)

In [ ]:
# combine significant proteins and identified metabolites together
result = pd.concat([dfer_mean, sigpro_stdrow], axis=0)
result=result.iloc[:,:3]
result

In [ ]:
kmeans=KMeans(n_clusters=4)
kmeans.fit(result[['Con','LPS','IL_4']])

result['kmeans']=kmeans.labels_
result

In [111]:
result.to_excel(r'D:\\project7_SMAD\\sig_multi_identified_dropdulpli_final.xlsx')

In [15]:
result2=pd.read_excel(r'D:\\project7_SMAD\\sig_multi_identified_dropdulpli_final.xlsx',index_col=0)

In [17]:
result3=result2.iloc[:,:3]

In [ ]:
sig_cluster3=result2.loc[result2['kmeans']==3]
sig_cluster2=result2.loc[result2['kmeans']==2]
sig_cluster1=result2.loc[result2['kmeans']==1]
sig_cluster0=result2.loc[result2['kmeans']==0]

# Create a dictionary of cluster names and corresponding data
cluster_data = {'multiCluster 1': result3.loc[list(sig_cluster0.index)],
                'multiCluster 2': result3.loc[list(sig_cluster1.index)],
                'multiCluster 3': result3.loc[list(sig_cluster2.index)],
                'multiCluster 4': result3.loc[list(sig_cluster3.index)]}
x=['Con','LPS','IL_4']
# Loop over the dictionary items to plot the data for each cluster
for title, data in cluster_data.items():
    plt.figure(figsize=(6, 5), edgecolor='#04253a')
    plt.rcParams['font.family'] = 'Arial'
    num1,num2=[],[]
    for index in data.index:
        if isinstance(index, float):
            plt.plot(x,list(data.loc[index]),color="#FFA500",alpha=0.8,linewidth=2,label='metabolites')
            num1.append(index)
        if isinstance(index, str):
            plt.plot(x,list(data.loc[index]),color="#7030A0",alpha=0.5,linewidth=2.2,label='proteins') 
            num2.append(index)
    print([len(num1),len(num2)])
    #plt.title(title, fontsize=24)
    #plt.xlabel('Treatment', fontsize=24)
    #plt.ylabel('Score', fontsize=24)
    plt.xticks(fontsize=26)
    plt.yticks(fontsize=26)
    plt.ylim(-1.7,1.7)
    plt.yticks([-1,  0,  1])
    plt.tick_params(axis='x', which='both', length=9)
    plt.savefig(r'D:\\project7_SMAD\\{}.svg'.format(title))
    plt.show()

In [3]:
# plot the heatmap for multiomic dataset

df=pd.read_excel(r'D:\\project7_SMAD\\sig_multi_identified_dropdulpli_final.xlsx',index_col=0,sheet_name="Sheet2")


In [ ]:
df_bb=df.T
df_heat=df_bb.iloc[:3,:]

plt.figure(figsize=(22,5),edgecolor="#04253a")
matplotlib.rcParams['font.family'] = "Arial"
sns.heatmap(df_heat,  vmin=-1.5, vmax=1.5,cmap='viridis',xticklabels=False,cbar_kws={"shrink": 1, 'ticks': [-2, -1, 0, 1, 2]})
plt.xticks([])
plt.yticks()
plt.tick_params(axis='y',length=6)
plt.yticks(rotation=90)
plt.yticks(fontsize=22)
plt.savefig(r'D:\\project7_SMAD\\figure_3_macro_clustering_analysis\\Figure3_heatmap.svg',dpi=1000)

In [ ]:
df_bb=df.T
df_heat=df_bb.iloc[:3,:]

plt.figure(figsize=(22,5),edgecolor="#04253a")
matplotlib.rcParams['font.family'] = "Arial"
sns.heatmap(df_heat,  vmin=-1.5, vmax=1.5,cmap='viridis',xticklabels=False,cbar_kws={"shrink": 1, 'ticks': [-2, -1, 0, 1, 2]})
plt.xticks([])
plt.yticks()
plt.tick_params(axis='y',length=6)
plt.yticks(rotation=90)
plt.yticks(fontsize=22)
#plt.savefig(r'D:\\project7_SMAD\\figure_3_macro_clustering_analysis\\Figure3_heatmap.tiff',dpi=600)

# plot typical metabolites and proteins here

In [119]:
treatment=['Con']*6+['LPS']*6+['IL-4']*6

def getdf(name='2/tr|A8XU21|A8XU21_MOUSE/sp|P97369|NCF4_MOUSE',df=dfpro_stdbyrow):    
    A=df.iloc[:,:6].loc[name].to_frame()
    A.columns=['Value']  
    B=df.iloc[:,6:12].loc[name].to_frame()
    B.columns=['Value']   
    C=df.iloc[:,12:18].loc[name].to_frame()
    C.columns=['Value'] 
    made=pd.concat([A,B,C])
    made['treatment'] = treatment
    return(made)

def plot_fig(listname=['proname'],df_name=dfpro_stdbyrow):
    for number in listname:
        plt.figure(figsize=(7,7))
        dataset=getdf(name=number,df=df_name)
        #palette = ['#9B3A4D', '#E2AE79', '#497EB2','#D0DCAA', '#8CBDA7', '#68AC57','#70A0AC']
        ax = sns.boxplot(x="treatment", y="Value",width=0.8, data=dataset,palette='viridis')
        ax = sns.stripplot(x="treatment", y="Value", data=dataset, color="black",size=12)
        ax.set_xlabel("")
        ax.set_ylabel("Score", fontsize=36)
        ax.tick_params(axis='both', labelsize=35)
        ax.set_xticklabels(ax.get_xticklabels(),rotation=0)
        plt.title(number)
        plt.show()

In [129]:
for item in dfpro_stdbyrow.index:
    if 'CMPK2' in item or 'RSAD2' in item or 'DDX58' in item or 'DHX58' in item:
        print(item)

2/tr|A2AP29|A2AP29_MOUSE/sp|Q6Q899|DDX58_MOUSE


In [134]:
for item in dfpro_stdbyrow.index:
    if 'MC' in item:
        print(item)

In [120]:
prolist=['1/sp|P56480|ATPB_MOUSE','2/tr|A2AP29|A2AP29_MOUSE/sp|Q6Q899|DDX58_MOUSE']



In [ ]:
plot_fig(listname=prolist,df_name=dfpro_stdbyrow)